In [1]:
from config import *
import pandas as pd
import requests
import json
import urllib
import dateutil.parser
import time
from datetime import datetime

In [34]:
# underlying stock ticker symbol
ticker = 'SPY'

# expiration date of desired option where last number '7' is the days until expiration
exp_date = '2023-01-20:7'

# strike price of desired option in str
strike_price = '395.0'

In [29]:
# Define TDAmeritrade Endpoint and Payload
tda_endpoint = 'https://api.tdameritrade.com/v1/marketdata/chains'
tda_payload = {'apikey': TDA_ID,
               'symbol': ticker,
               'contractType': 'CALL',
               'includeQuotes': 'TRUE',
               'strategy': 'SINGLE',
               'expMonth': 'ALL',
               "isDelayed": 'true'}

# define alpaca endpoint and url
HEADERS = {'APCA-API-KEY-ID': ALPACA_KEY_ID, 'APCA-API-SECRET-KEY': ALPACA_SECRET_KEY}
alpaca_endpoint = 'https://paper-api.alpaca.markets'
account_url = '{}/v2/account'.format(alpaca_endpoint)
orders_url = '{}/v2/orders'.format(alpaca_endpoint)
positions_url = '{}/v2/positions'.format(alpaca_endpoint)
portfolio_url = '{}/v2/account/portfolio/history'.format(alpaca_endpoint)

In [30]:
# this function will get the profit/loss on the stock position in alpaca
def get_pnl():
    data = {
        'period': '1D',
        'timeframe': '1D'
    }
    r = requests.get(portfolio_url, headers=HEADERS)
    return json.loads(r.content)


In [35]:
# this returns how much you sold the option for in nominal dollars
r = requests.get(url=tda_endpoint, params=tda_payload)
tda_data = json.loads(r.content)
option_sell_price = tda_data['callExpDateMap'][exp_date][strike_price][0]['bid'] * 100

pnl_list = []

In [27]:
# get the profit/loss for the stock position on alpaca
alpaca_pnl = get_pnl()['profit_loss'][-1]
print(alpaca_pnl)

111


In [37]:
tda_data['callExpDateMap'][exp_date][strike_price][0]['delta'] * -100

-58.9